<font size=4>This code is used to export mosaicked satellite imagery</font>

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal
import ee
import geemap
import numpy as np
import matplotlib.pyplot as plt
import os

<font size=4>Authenticate and initialize your Google Earth Engine account</font>

In [5]:
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AX4XfWglnPaXF2Tr5dvnl4ftI0kzxo7IfSX8K9H8MYRhqa5n4Lv_0oZTfmY

Successfully saved authorization token.


timeout: timed out

<font size=4>Basic functions sentienl-2 imagery preprocessing</font>

In [ ]:
def decodeQA60(img):
    qa60 = img.select('QA60')#.updateMask(img.select('B2'))
    cloudBitMask = qa60.bitwiseAnd(ee.Number(2).pow(10).int())
    cirrusBitMask = qa60.bitwiseAnd(ee.Number(2).pow(11).int())
    clear = cloudBitMask.eq(0).And(cirrusBitMask.eq(0)).rename(['PXQA60_CLEAR']).toInt()
    clear = clear.updateMask(clear)
    return img.addBands([clear])

def applyCloudmask(img):
    clearmask = img.select('PXQA60_CLEAR')
    return img.updateMask(clearmask)

def cutoverlap(img): 
    geo1 = img.geometry();
    geo2 = img.translate(-9900.0, -9900, 'meters').geometry();
    geo = geo1.intersection(geo2)
    return ee.Algorithms.If(geo.geometries().length(), img.clip(geo), None);

def get_orbit_num(imgCol):
    return imgCol.aggregate_array('SENSING_ORBIT_NUMBER').distinct()

def cloudfree(date):
#     aoi = ee.FeatureCollection("TIGER/2018/States").filterMetadata('NAME', 'equals', 'Iowa').geometry()
#     aoi = aoi.bounds()
    increase = ee.Number(date)
    start = ee.Date('2018-04-01').advance(increase.multiply(5), 'day')
    end = ee.Date('2018-04-06').advance(increase.multiply(5), 'day')
    s2col = ee.ImageCollection("COPERNICUS/S2").filterDate(start, end).filterBounds(aoi).map(cutoverlap, True)\
.filterMetadata('MGRS_TILE', 'equals', '14TQM')
    orbit = get_orbit_num(s2col)
    def mosaic(i):
        ds = s2col.filter(ee.Filter.eq('SENSING_ORBIT_NUMBER', i))
        return ds.mosaic()  ## revised by ZJ
    mosaiced = orbit.map(mosaic)
    cloud_mask =  ee.ImageCollection(mosaiced).map(decodeQA60).map(applyCloudmask)
    return cloud_mask.mosaic()  ##revised by ZJ


In [ ]:
aoi = ee.FeatureCollection("TIGER/2018/States").filterMetadata('NAME', 'equals', 'Iowa').geometry()
aoi = aoi.bounds()
CDL = ee.ImageCollection("USDA/NASS/CDL").filterDate('2017-01-01', '2017-12-31')

In [ ]:
for i in range(0, 1):
    intervals = ee.List.sequence(i, i)
    countCol = intervals.map(cloudfree)
    Map.addLayer(ee.Image(countCol.get(0)))
#     band = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']
    band = ['B5', 'B11']
    geemap.ee_export_image_to_drive(ee.Image(countCol.get(0)).select(band), \
                         description="ia_2018_" + str(i), \
                         folder='GEE_IA_2018', \
                         max_pixels=1e13, \
                         region=ee.ImageCollection("COPERNICUS/S2").filterMetadata('MGRS_TILE', 'equals', '14TQM').first().geometry().bounds(), \
                         crs='EPSG:4326', \
                         scale=30)
    Map

In [12]:
geemap.ee_export_image_to_drive(ee.Image(CDL.first()).select('cropland'), \
                     description="IA_CDL_2017", \
                     folder='GEE_IA_2018', \
                     max_pixels=1e13, \
                     region=ee.ImageCollection("COPERNICUS/S2").filterMetadata('MGRS_TILE', 'equals', '14TQM').first().geometry().bounds(), \
#                      region=mc, \
                     crs='EPSG:4326', \
                     scale=30)
# arr = geemap.ee_to_numpy(ee.Image(test))

Exporting IA_CDL_2017 ...
